## Setup

In [20]:
import os
import subprocess
import shutil

## Create Model

In [21]:

def create_model(model_name):
    command = f"abaqus cae noGUI=.\{model_name}.py"
    try:
        subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Abaqus job completed successfully.")

        # Define the source and destination paths
        source_path = 'cantilever_beam.inp'
        destination_path = 'src/abaqus_files/cantilever_beam.inp'

        # Move the file
        shutil.move(source_path, destination_path)
        print("File moved successfully.")

    except subprocess.CalledProcessError as e:
        print("Error during Abaqus job execution.")
        print("Error output:", e.stderr)


In [22]:
create_model("src/utils/create_model")

Abaqus job completed successfully.
File moved successfully.


In [4]:
def run_abaqus(job_name, input_file, target_directory):
    # Run the Abaqus job
    command = f"abaqus job={job_name} input={input_file}"
    try:
        subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Abaqus job completed successfully.")
        
        # Define the source directory
        source_directory = os.getcwd()  # Assumes Abaqus outputs files in the current working directory

        # Loop through files in the source directory
        for file_name in os.listdir(source_directory):
            source_path = os.path.join(source_directory, file_name)
            if os.path.isfile(source_path) and file_name.startswith(job_name):
                target_path = os.path.join(target_directory, file_name)
                shutil.move(source_path, target_path)
                print(f"Moved {file_name} to {target_directory}")
    except subprocess.CalledProcessError as e:
        print("Error during Abaqus job execution.")
        print("Error output:", e.stderr)

In [5]:
# Define job parameters
job_name = "cantilever_beam"
input_file = "src/abaqus_files/cantilever_beam.inp"
target_directory = "src/abaqus_files"  # Directory to move .odb files to

# Run the function with the updated parameters
run_abaqus(job_name, input_file, target_directory)

Abaqus job completed successfully.
Moved cantilever_beam.com to src/abaqus_files
Moved cantilever_beam.dat to src/abaqus_files
Moved cantilever_beam.log to src/abaqus_files
Moved cantilever_beam.msg to src/abaqus_files
Moved cantilever_beam.odb to src/abaqus_files
Moved cantilever_beam.prt to src/abaqus_files
Moved cantilever_beam.sta to src/abaqus_files


In [6]:
def run_extract_stress(extract_stress_path, target_directory):
    command = f"abaqus python {extract_stress_path}.py"
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Abaqus job completed successfully.")
        print("Output:", result.stdout)

        # Check if the target file exists
        source_file = os.path.join(os.getcwd(), "max_vm_stress.txt")
        if os.path.exists(source_file):
            # Ensure the target directory exists
            os.makedirs(target_directory, exist_ok=True)

            # Move the file to the target directory
            target_file = os.path.join(target_directory, "max_vm_stress.txt")
            shutil.move(source_file, target_file)
            print(f"Moved {source_file} to {target_file}")

    except subprocess.CalledProcessError as e:
        print("Error during Abaqus job execution.")
        print("Error output:", e.stderr)

In [7]:
extract_stress_path = "src/utils/retrieve_vm_stress"
target_directory = "src/abaqus_files"
run_extract_stress(extract_stress_path,target_directory )

Abaqus job completed successfully.
Output: Maximum Mises stress in the cantilever beam is 136533456.000000

Moved c:\Users\Farhad.Davaripour\Repositories\ReAct_agent_integration_w_FEA\max_vm_stress.txt to src/abaqus_files\max_vm_stress.txt


In [15]:
def parse_stress_data(file_path):
    parsed_data = []
    
    # Open and read the file line by line
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove any trailing whitespace
            
            # Check if the line contains data of interest
            if "Mises stress" in line:
                mises_stress = float(line.split("is ")[-1].strip())
                # Append the parsed data as a dictionary
                parsed_data.append({
                    "von-Mises Stress": f"{round(mises_stress/10e6, 2)} MPA"
                })
    
    return parsed_data

In [16]:
# Parse the data file
data = parse_stress_data('src/abaqus_files/max_vm_stress.txt')

# Display parsed data
for entry in data:
    print(entry)

{'von-Mises Stress': '13.65 MPA'}
